In [1]:
import numpy as np

from strategy import RSIRisingFalling

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=3000,
)

timestamps = candles[:, CandleBodyType.Timestamp]
datetimes = timestamps.astype("datetime64[ms]")
open = candles[:, CandleBodyType.Open]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]
volume = candles[:, CandleBodyType.Volume]

In [3]:
short_strat = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50, 76, 5),
)
short_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
short_strat.plot_signals(candles=candles)

In [4]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(35, 61, 5),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_strat.plot_signals(candles=candles)

In [5]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=False,
    z_or_e_type=None,
)

In [6]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 48
Total combinations of settings to test: 288
Total candles: 3,000
Total candles to test: 864,000


In [7]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(20)

ind_set_idx  dos_index  total_trades  wins  losses  gains_pct  win_rate  \
211            4         19          37.0    17      20    324.974    45.946   
210            4         18          37.0    17      20    324.974    45.946   
209            4         17          37.0    17      20    324.974    45.946   
208            4         16          37.0    17      20    324.974    45.946   
183            3         39          17.0     7      10    633.905    41.176   
182            3         38          17.0     7      10    633.905    41.176   
181            3         37          17.0     7      10    633.905    41.176   
180            3         36          17.0     7      10    633.905    41.176   
287            5         47           9.0     4       5    247.428    44.444   
286            5         46           9.0     4       5    247.428    44.444   
285            5         45           9.0     4       5    247.428    44.444   
284            5         44           9.0     4       5    247.428    44.444   
139            2         43          23.0     8      15    500.918    34.783   
136            2         40          23.0     8      15    500.918    34.783   
138            2         42          23.0     8      15    500.918    34.783   
137            2         41          23.0     8      15    500.918    34.783   
131            2         35          33.0     9      24    445.744    27.273   
130            2         34          33.0     9      24    445.744    27.273   
129            2         33          33.0     9      24    445.744    27.273   
128            2         32          33.0     9      24    445.744    27.273   

     qf_score  fees_paid  total_pnl  ending_eq  
211     0.885    507.643   3249.735   4249.735  
210     0.885    507.643   3249.735   4249.735  
209     0.885    507.643   3249.735   4249.735  
208     0.885    507.643   3249.735   4249.735  
183     0.880    406.491   6339.051   7339.051  
182     0.880    406.491   6339.051   7339.051  
181     0.880    406.491   6339.051   7339.051  
180     0.880    406.491   6339.051   7339.051  
287     0.879     62.366   2474.277   3474.277  
286     0.879     62.366   2474.277   3474.277  
285     0.879     62.366   2474.277   3474.277  
284     0.879     62.366   2474.277   3474.277  
139     0.860    453.708   5009.181   6009.181  
136     0.860    453.708   5009.181   6009.181  
138     0.860    453.708   5009.181   6009.181  
137     0.860    453.708   5009.181   6009.181  
131     0.856    719.536   4457.442   5457.442  
130     0.856    719.536   4457.442   5457.442  
129     0.856    719.536   4457.442   5457.442  
128     0.856    719.536   4457.442   5457.442

In [8]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    dos_index=17,
    ind_set_index=4,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 5,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.005,
    sl_based_on_lookback = 20,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.02,
)
IndicatorSettingsArrays(
    rsi_is_above = nan,
    rsi_is_below = 55.0,
    rsi_length = 14,
)


In [9]:
order_records_df

ind_set_idx  or_set_idx  bar_idx      timestamp            datetime  \
0              4          17       51  1692554400000 2023-08-20 18:00:00   
1              4          17       53  1692561600000 2023-08-20 20:00:00   
2              4          17       59  1692583200000 2023-08-21 02:00:00   
3              4          17       63  1692597600000 2023-08-21 06:00:00   
4              4          17       72  1692630000000 2023-08-21 15:00:00   
..           ...         ...      ...            ...                 ...   
163            4          17     2747  1702260000000 2023-12-11 02:00:00   
164            4          17     2749  1702267200000 2023-12-11 04:00:00   
165            4          17     2752  1702278000000 2023-12-11 07:00:00   
166            4          17     2754  1702285200000 2023-12-11 09:00:00   
167            4          17     2762  1702314000000 2023-12-11 17:00:00   

       order_status    equity  available_balance  cash_borrowed  cash_used  \
0       EntryFilled  1000.000            957.637       2402.058     42.363   
1       EntryFilled  1000.000            873.622       7086.464    126.378   
2       EntryFilled  1000.000            748.032      13969.034    251.968   
3       EntryFilled  1000.000            577.946      23369.368    422.054   
4    StopLossFilled   878.998            878.998            NaN        NaN   
..              ...       ...                ...            ...        ...   
163  StopLossFilled  4249.735           4249.735            NaN        NaN   
164     EntryFilled  4249.735           4110.402       2218.255    139.333   
165     EntryFilled  4249.735           3828.871       6701.299    420.864   
166     EntryFilled  4249.735           3411.672      13125.565    838.063   
167     EntryFilled  4249.735           2845.389      21936.985   1404.346   

     ...  entry_size_usd  entry_price  exit_price  position_size_asset  \
0    ...        2444.421      26111.3         NaN                0.094   
1    ...        2366.363      26122.0         NaN                0.185   
2    ...        2323.754      26131.8         NaN                0.274   
3    ...        2687.850      26085.5         NaN                0.377   
4    ...             NaN          NaN     25822.0                  NaN   
..   ...             ...          ...         ...                  ...   
163  ...             NaN          NaN     42612.1                  NaN   
164  ...        2357.588      42312.9         NaN                0.056   
165  ...        2546.320      42135.2         NaN                0.116   
166  ...        2358.421      42246.3         NaN                0.172   
167  ...        2953.437      41831.6         NaN                0.243   

     position_size_usd  realized_pnl  sl_pct  sl_price  tp_pct  tp_price  
0             2444.421           NaN     1.1   25822.0     2.5   26775.9  
1             4810.784           NaN     1.1   25822.0     2.6   26789.9  
2             7134.538           NaN     1.1   25822.0     2.6   26801.7  
3             9822.388           NaN     1.1   25822.0     2.5   26773.5  
4                  NaN      -121.002     NaN       NaN     NaN       NaN  
..                 ...           ...     ...       ...     ...       ...  
163                NaN      -575.419     NaN       NaN     NaN       NaN  
164           2357.588           NaN     5.3   40083.0    10.9   46911.0  
165           4903.908           NaN     5.1   40083.0    10.5   46668.3  
166           7262.329           NaN     5.1   40083.0    10.6   46723.1  
167          10215.766           NaN     4.9   40083.0    10.0   46387.7  

[168 rows x 27 columns]